In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import shap
import matplotlib.pyplot as plt
import pickle as pkl
from joblib import load#,dump

In [25]:
ur_df = pd.read_csv('usr_emo_lean.csv.gz').dropna().query('n_emo > 0')

# Every user without emotiveness was excluded while cleaning.
# Proof:
# print(df.query('w_emotiveness == 0'))

# The target variable is 'is_questionable'.
#Y = np.where(df['is_questionable'].astype(int)>0,1,0)
# The features are the 8 plutchik emotions, plus the 4 toxicity percentages
#X = df[['anger','anticipation','disgust','fear','joy','sadness','surprise','trust']]

In [29]:
#model = load(os.path.join(os.listdir()[0], 'smote_model.joblib'))




KeyError: 0

In [38]:
for dir in os.listdir():
    if os.path.isdir(dir):
        (min_comments, min_emo) = [int(os.listdir()[0].split('_')[i]) for i in [0,3] ]
        df = ur_df.query("n_comments >= @min_comments and n_emo >= @min_emo")
        Y = np.where(df['is_questionable'].astype(int)>0,1,0)
        X = df[['anger','anticipation','disgust','fear','joy','sadness','surprise','trust']]
        QX = df.query("is_questionable > 0.5")[['anger','anticipation','disgust','fear','joy','sadness','surprise','trust']]
        QY = np.where(df.query("is_questionable > 0.5")['is_questionable'].astype(int)>0,1,0)
        RX = df.query("is_questionable <= 0.5")[['anger','anticipation','disgust','fear','joy','sadness','surprise','trust']]
        RY = np.where(df.query("is_questionable <= 0.5")['is_questionable'].astype(int)>0,1,0)
        #model = load(os.path.join(dir, 'smote_model.joblib'))
        oversample = SMOTE(random_state=1518)

        #Augment the data points with SMOTE
        over_X, over_Y = oversample.fit_resample(X, Y)

        # Instanciate a standard Random Forest Regressor
        SMOTE_model = RandomForestClassifier(random_state=1518)
        # Fit the data
        SMOTE_model.fit(over_X,over_Y)
        with open(str(os.path.join(dir,'Qshap.pkl')), 'rb') as f:
            Qshap = pkl.load(f)
        shap_obj = shap.TreeExplainer(SMOTE_model)(QX)
        
        #with open(str(os.path.join(dir,'Rshap.pkl')), 'rb') as f:
        #    Rshap = pkl.load(f)
        #shap.summary_plot(Qshap, QX, show=True)
        shap.plots.beeswarm(shap_obj)

        #plt.savefig(os.path.join(results_path, 'Qshap.pdf'))
        break

ValueError: The beeswarm plot does not support plotting explanations with instances that have more than one dimension!